In [52]:
import soundfile as sf

start_sec = 1

audio, sr = sf.read(
    "/fsx/behavioural_computing_data/face_generation_data/CelebV-Text/audio/665/kcBjMJSaMX4_9_0_0_25.wav",
    start=0,
    # frames=16000*1,
    always_2d=True,
)  # e.g (16000, 1)

print(audio.shape, sr)

(16000, 1) 16000


In [53]:
from IPython.display import Audio

Audio(audio[:, 0], rate=sr)

In [3]:
import numpy as np

np.load("/fsx/behavioural_computing_data/face_generation_data/AA_processed/part_20/landmarks_crop/D088_C030_08047Q_001_output_keypoints.npy", allow_pickle=True).shape

(2417, 68, 2)

In [9]:
import math

video_rate = 60
audio_rate = sr
a2v_ratio = video_rate / float(audio_rate)
samples_per_frame = math.ceil(1 / a2v_ratio)
print(samples_per_frame, a2v_ratio, audio.shape[0])

# Knowing the number of samples per frame, we can calculate the number of frames in the audio
audio_frames = audio.shape[0] // samples_per_frame
print(audio_frames)

267 0.00375 603819
2261


In [31]:
from torchvision.io import read_video
import torch

video_path = "/fsx/behavioural_computing_data/face_generation_data/AA_processed/part_25/video_crop/D271_C045_0921NI_001_output_output.mp4"
resolution = 64
device = torch.device("cuda")

video, _, info = read_video(video_path, pts_unit="sec", output_format="TCHW")

In [44]:
def convert_indexes(indexes_25fps, fps_from=25, fps_to=60):
    ratio = fps_to / fps_from
    indexes_60fps = [int(index * ratio) for index in indexes_25fps]
    return indexes_60fps


indexes = [start_sec * 25 + i for i in range(50)]
indexes = convert_indexes(indexes, fps_from=25, fps_to=60)
print(indexes, len(indexes))

video_seg = video[indexes]
print(video_seg.shape)

[1800, 1802, 1804, 1807, 1809, 1812, 1814, 1816, 1819, 1821, 1824, 1826, 1828, 1831, 1833, 1836, 1838, 1840, 1843, 1845, 1848, 1850, 1852, 1855, 1857, 1860, 1862, 1864, 1867, 1869, 1872, 1874, 1876, 1879, 1881, 1884, 1886, 1888, 1891, 1893, 1896, 1898, 1900, 1903, 1905, 1908, 1910, 1912, 1915, 1917] 50
torch.Size([50, 3, 512, 512])


In [45]:
import torch
import torchvision
from torchvision.io import write_video
from IPython.display import HTML
import numpy as np
from base64 import b64encode

# Assuming video_tensor is your PyTorch tensor
# Shape: (num_frames, channels, height, width)
video_tensor = video_seg  # Replace with your actual tensor

# Set the frames per second (fps)
fps = 25

# Convert the tensor to uint8 (0-255 range) if necessary
# Assuming video_tensor is normalized between 0 and 1
video_tensor = video_tensor.byte()

# Write the video using torchvision
output_file = "output_video.mp4"
write_video(output_file, video_tensor.permute(0, 2, 3, 1), fps=fps)


# Function to display video in Jupyter Notebook
def display_video(file_path):
    # Convert the video file to a base64 string
    video_file = open(file_path, "rb").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

    # Display the video
    return HTML(f"""
        <video width="512" height="512" controls>
            <source src="{video_url}" type="video/mp4">
        </video>
    """)


# Display the video
display_video(output_file)